### This file is associated with my research project testing the hypothesis that prescriptive power of non-physician is associated with more opioid prescriptions. 
### I use raw arcos dataset provided by washington post. This data follows each painkiller pill in the years 2006-2012 across all states in the USA.
### As expected, it is a massive dataset of 17GB (or 17 million kB) I have downloaded and saved on my PC, and as such I will not be able to process it using my RAM. I could use the arcos API provided by wapo to perform the same task, but in my experience their server was always to busy to even perform simple tasks.
### My goal in this file is to use the panel data I collapsed from the raw file and a few other sources for further testing for regression discontinuity at the state-level. 

Steps in this analysis:
1. Use panel.csv file to make a dataframe (df) with months and state as indices.
1. Model panel regressions for testing the dependence of drug quantity (Hydroxycodine, Oxycodine, and Both) on various law changes.

(c) Arsh Singh, Dec 2022

In [1]:
import pandas as pd

In [2]:
folder='D:/arcos_all_washpost/'

In [3]:
file='panel.csv'
path=folder+file
panel_temp= pd.read_csv(path, usecols = ['STATE', 'months', 'np', 'np_no_md','pa','any','hydrocodone_per_cap','hydrocodone','oxycodone_per_cap','oxycodone','both_per_cap','both'])
panel_temp['trend_np']=panel_temp['np']*panel_temp['months']/panel_temp['months'].mean()
panel_temp['trend_np_no_md']=panel_temp['np_no_md']*panel_temp['months']/panel_temp['months'].mean()
panel_temp['trend_pa']=panel_temp['pa']*panel_temp['months']/panel_temp['months'].mean()
panel_temp['trend_any']=panel_temp['any']*panel_temp['months']/panel_temp['months'].mean()
file='panel_trend.csv'
path=folder+file
panel_temp.to_csv(path,index=None)

In [4]:
file='panel_trend.csv'
path=folder+file
panel= pd.read_csv(path, usecols = ['STATE', 'months', 'np', 'np_no_md','pa', 'any',
                                    'hydrocodone_per_cap','hydrocodone',
                                    'oxycodone_per_cap','oxycodone',
                                    'both_per_cap','both',
                                    'trend_np','trend_np_no_md','trend_pa','trend_any'
                                   ],index_col=['STATE','months'])

In [5]:
panel[['trend_np','trend_np_no_md','trend_pa','trend_any']].describe()

,trend_np,trend_np_no_md,trend_pa,trend_any
count,2804.000000,2804.000000,2804.000000,2804.000000
mean,0.659086,0.146237,0.570285,0.693346
std,0.474131,0.353432,0.495153,0.461242
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.999300,0.000000,0.998927,0.999466
75%,1.000420,0.000000,1.000254,1.000503
max,1.001457,1.001457,1.001457,1.001457


In [6]:
months= panel.index.get_level_values('months').to_list()
panel['months'] = pd.Categorical(months)

In [7]:
# FE und RE model
import statsmodels.api as sm
from linearmodels import PanelOLS
from linearmodels import RandomEffects

In [8]:
exog_vars = ['np_no_md','np','pa','trend_np_no_md','trend_np','trend_pa']
exog = sm.add_constant(panel[exog_vars])
endog = panel['both_per_cap']

# fixed effects model
model_fe = PanelOLS(endog, exog, entity_effects = True) 
fe_res = model_fe.fit() 
print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:           both_per_cap   R-squared:                        0.1772
Estimator:                   PanelOLS   R-squared (Between):             -0.1468
No. Observations:                2804   R-squared (Within):               0.1772
Date:                Wed, Jan 04 2023   R-squared (Overall):             -0.0886
Time:                        06:42:32   Log-likelihood                   -5232.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      98.884
Entities:                          44   P-value                           0.0000
Avg Obs:                       63.727   Distribution:                  F(6,2754)
Min Obs:                       2.0000                                           
Max Obs:                       71.000   F-statistic (robust):             98.884
                            

In [9]:
exog_vars = ['np_no_md','np','pa','trend_np_no_md','trend_np','trend_pa']
exog = sm.add_constant(panel[exog_vars])
endog = panel['both_per_cap']

#random effects model
model_re = RandomEffects(endog, exog) 
re_res = model_re.fit() 
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:           both_per_cap   R-squared:                        0.1715
Estimator:              RandomEffects   R-squared (Between):             -0.1118
No. Observations:                2804   R-squared (Within):               0.1771
Date:                Wed, Jan 04 2023   R-squared (Overall):             -0.0801
Time:                        06:42:32   Log-likelihood                   -5260.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      96.476
Entities:                          44   P-value                           0.0000
Avg Obs:                       63.727   Distribution:                  F(6,2797)
Min Obs:                       2.0000                                           
Max Obs:                       71.000   F-statistic (robust):             97.361
                            